In [39]:
from tensorflow.keras import layers, models, Input

import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFont, ImageDraw
import random
from keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

data_directory = 'sketch_small'
TARGET_SIZE = (128, 128, 3)
EPOCHS = 10
BATCH_SIZE = 32

In [40]:
def create_pairs(directory):
    #directory is the path of the dataset (main folder)
    #folders are classes in dataset
    pairs = []
    labels = []
    
    folders = os.listdir(directory)
    for folder_name in folders:
        folder_path = os.path.join(directory, folder_name)
        
        if os.path.isdir(folder_path):
            images = os.listdir(folder_path)
            
            folder_length = len(images)
            for i in range(folder_length):
                for j in range(folder_length):
                    if i != j:
                        image_path = os.path.join(folder_path, images[i])
                        pairs.append([image_path, os.path.join(folder_path, images[j])])
                        labels.append(1)#positive pairs
                        
                        dif_folder = random.choice([x for x in folders if x != folder_name])
                        dif_folder_path = os.path.join(directory, dif_folder)
                        dif_image_path = os.path.join(dif_folder_path, random.choice(os.listdir(dif_folder_path)))

                        pairs.append([image_path, dif_image_path])
                        labels.append(0)#negative pairs
                        
    return np.array(pairs), np.array(labels) 

In [41]:
def process_images(pairs, target_size):
    images = []
     
    for pair in pairs:
      
        img1 = np.array(load_img(pair[0], target_size=target_size))
        img2 = np.array(load_img(pair[1], target_size=target_size))

        images.append((img1, img2))
        
    return images

In [42]:
def prepare_data(directory):
    X, y = create_pairs(directory)
    X = process_images(X, TARGET_SIZE)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    return np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

X_train, X_test, y_train, y_test = prepare_data(data_directory)  

In [43]:
def create_siamese(input_shape=TARGET_SIZE):  # Assuming RGB images with height and width of 128 pixels
    # Define the base CNN architecture
    base_model = models.Sequential()
    base_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    base_model.add(layers.MaxPooling2D((2, 2)))
    base_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    base_model.add(layers.MaxPooling2D((2, 2)))
    base_model.add(layers.Flatten())
    base_model.add(layers.Dense(128, activation='relu'))

    # Define the left input and right input
    left_input = layers.Input(shape=input_shape)
    right_input = layers.Input(shape=input_shape)

    # Encode each input using the base CNN
    encoded_left = base_model(left_input)
    encoded_right = base_model(right_input)

    # Compute the L1 distance between the encoded outputs
    l1_distance = layers.Lambda(lambda tensors: abs(tensors[0] - tensors[1]))([encoded_left, encoded_right])

    # Add a dense layer with sigmoid activation to output similarity score
    prediction = layers.Dense(1, activation='sigmoid')(l1_distance)

    # Connect the inputs with the prediction
    siamese_model = models.Model(inputs=[left_input, right_input], outputs=prediction)

    return siamese_model


In [46]:
X_train_left = X_train[:, 0]
X_train_right = X_train[:, 1]
X_train_left.shape

(17850, 128, 128, 3)

In [50]:
siamese_model = create_siamese()
siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
siamese_model.fit([X_train_left, X_train_right], y_train, epochs=2, batch_size=32, validation_split=0.2)

Epoch 1/2
447/447 ━━━━━━━━━━━━━━━━━━━━ 139s 306ms/step - accuracy: 0.5003 - loss: 8.1974 - val_accuracy: 0.4950 - val_loss: 0.6989
Epoch 2/2
447/447 ━━━━━━━━━━━━━━━━━━━━ 136s 305ms/step - accuracy: 0.5004 - loss: 0.6948 - val_accuracy: 0.5000 - val_loss: 0.6928


In [67]:
im1 = np.array(X_train_left[120])
im2 = np.array(X_train_right[1224])

im1 = im1.reshape(1, im1.shape[0], im1.shape[1], im1.shape[2])
im2 = im2.reshape(1, im2.shape[0], im2.shape[1], im2.shape[2])

im2.shape

(1, 128, 128, 3)

In [69]:
p = siamese_model.predict([im1, im2])
print(p)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
[[0.4999949]]
0
